<a href="https://colab.research.google.com/github/Tommy-Praise/Llama-Chatbot-with-Sentiment-Analysis-Integration/blob/main/Llama_Chatbot_with_Sentiment_Analysis_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -q accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub
!pip install -q gradio

In [2]:
# Import necessary libraries
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch
import gradio as gr

In [ ]:
# Hugging Face Authentication using your access token
login(token="Your access token")

In [4]:
# Defining the path for the CSV file
csv_file = 'qa_dataset.csv'

# Check if the CSV file exists; if not, create it with initial data
if not os.path.exists(csv_file):
    qa_data = {
        'question': ["What is the name of Julius Magellan's dog?", "Who is Julius Magellan's dog?"],
        'answer': ["The name of Julius Magellan's dog is Sparky", "Julius Magellan's dog is called Sparky"]
    }
    qa_df = pd.DataFrame(qa_data)
    qa_df.to_csv(csv_file, index=False)
else:
    # Loading the existing CSV file into a DataFrame
    qa_df = pd.read_csv(csv_file)

In [ ]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

In [ ]:
# Initialize sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

In [7]:
# Initialize the text generation pipeline (Llama 2)
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024
)

In [8]:
# Determining sentiment and adjusting chatbot response
def adjust_response_based_on_sentiment(response, sentiment):
    if sentiment == 'POSITIVE':
        return f"I'm glad you're feeling positive! Here's your answer: {response}"
    elif sentiment == 'NEGATIVE':
        return f"I'm sorry to hear that. Let me assist you with the answer: {response}"
    else:
        return response  # Neutral or mixed sentiment, return the response as is

In [12]:
# Function to answer a question with sentiment analysis using the model
def gradio_chat_interface_with_sentiment(question):
    global qa_df

    # Detect the sentiment of the user's question
    sentiment_result = sentiment_pipeline(question)
    sentiment = sentiment_result[0]['label']

    # Check if the question is in the QA dataset
    answer = qa_df[qa_df['question'].str.lower() == question.lower()]['answer']

    if not answer.empty:
        response = answer.iloc[0]
        print(f"Answer from QA dataset: {response}")
    else:
        # Use Llama 2 to generate an answer
        response = llama_pipeline(question, max_length=150, do_sample=True)[0]['generated_text']
        response = response.replace(f"Answer: {question}", "").strip()
        print(f"Answer from Llama 2: {response}")

        # Add the new QA pair to the dataset
        new_row = pd.DataFrame({'question': [question], 'answer': [response]})
        qa_df = pd.concat([qa_df, new_row], ignore_index=True)
        qa_df.to_csv(csv_file, index=False)

    # Adjust the response based on sentiment
    adjusted_response = adjust_response_based_on_sentiment(response, sentiment)
    print(f"Sentiment: {sentiment}")
    return adjusted_response

In [13]:
# Create a Gradio Interface
interface = gr.Interface(
    fn=gradio_chat_interface_with_sentiment,
    inputs="text",
    outputs="text",
    title="Llama 2 Chatbot with Sentiment Analysis",
    description="Ask a question, and the chatbot will respond using a pre-defined QA dataset or Llama 2. It will adjust its response based on the sentiment of your question."
)

# Launch the Gradio Interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5bd0e1559b2d23b710.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Gradio Interface
def gradio_chat_interface_with_sentiment(question):
    global qa_df
    sentiment_result = sentiment_pipeline(question)
    sentiment = sentiment_result[0]['label']

    # Check if the question is in the QA dataset
    answer = qa_df[qa_df['question'].str.lower() == question.lower()]['answer']

    if not answer.empty:
        response = answer.iloc[0]
        adjusted_response = adjust_response_based_on_sentiment(response, sentiment)
        return f"Answer from QA dataset: {adjusted_response}"
    else:
        # Generate an answer using Llama 2
        response = llama_pipeline(question, max_length=150, do_sample=True)[0]['generated_text']
        response = response.replace(f"Answer: {question}", "").strip()

        # Add the new question-answer pair to the dataset
        new_row = pd.DataFrame({'question': [question], 'answer': [response]})
        qa_df = pd.concat([qa_df, new_row], ignore_index=True)
        qa_df.to_csv(csv_file, index=False)

        adjusted_response = adjust_response_based_on_sentiment(response, sentiment)
        return f"Answer from Llama 2: {adjusted_response} \n(New QA pair added to the dataset.)"
